In [1]:
import pandas as pd   
from matplotlib import pyplot as plt
from numpy import mean
import time

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import RobustScaler

import warnings 
warnings.filterwarnings('ignore')


from sklearn.linear_model import LinearRegression

In [2]:
df1 = pd.read_csv("train.csv")
#df1.head()
df2 = pd.read_csv('test.csv')
#df2

In [3]:
#from df1 drop sub_are
df1 = df1.drop(columns=['sub_area'])


In [4]:
#from df2 drop sub_area and row ID 
df2 = df2.drop(columns=['sub_area', 'row ID'])

In [5]:
df_onehot = pd.get_dummies(df1)
df_onehot.dtypes
test_onehot = pd.get_dummies(df2)

In [6]:
X = df_onehot.drop(columns=['price_doc'])
print(X.shape)
y = df_onehot[['price_doc']]




(181507, 287)


In [7]:
#do robust scaling
scaler = RobustScaler()
scaled_df2 = scaler.fit_transform(test_onehot)
scaled_df1 = scaler.fit_transform(X)

In [ ]:
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and test data
df_onehot_scaled = scaler.fit_transform(df1)
test_onehot_scaled = scaler.transform(df2)

In [264]:
import statsmodels.api as sm
import pandas as pd



# Add a constant term to the feature matrix
X_with_const = sm.add_constant(X)

# Fit a linear regression model
model = sm.OLS(y, X_with_const).fit()

# Get p-values for each feature
p_values = model.pvalues[1:]  # Exclude the constant term

# Set your desired threshold for p-value
threshold = 0.025

# Filter features based on p-value
selected_features = p_values[p_values < threshold].index

# Display selected features
print("Selected Features:")
print(selected_features)
X_selected = X[selected_features]

Selected Features:
Index(['full_sq', 'life_sq', 'floor', 'raion_popul', 'children_preschool',
       'preschool_education_centers_raion', 'children_school',
       'school_education_centers_top_20_raion', 'healthcare_centers_raion',
       'university_top_20_raion',
       ...
       'detention_facility_raion_no', 'water_1line_no', 'water_1line_yes',
       'big_road1_1line_no', 'big_road1_1line_yes', 'railroad_1line_yes',
       'ecology_excellent', 'ecology_good', 'ecology_no data',
       'ecology_satisfactory'],
      dtype='object', length=170)


In [ ]:
#give me code for feature importance using linear regression
model = LinearRegression()
model.fit(X, y)
# get importance
importance = model.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

# drop features with low importance
threshold = 0  # Adjust the threshold as needed
selected_features = [feature for feature, score in zip(X.columns, importance) if score >= threshold]
X_selected = X[selected_features]

# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()



In [9]:
ols_reg = LinearRegression()
sfs = SequentialFeatureSelector(ols_reg, direction='forward', n_features_to_select=25)
sfs.fit(scaled_df1, y)
X_selected = sfs.transform(scaled_df1)
# Print the names of the selected features
print("Selected Features:", sfs.get_feature_names_out())




Selected Features: ['x0' 'x1' 'x2' 'x11' 'x42' 'x51' 'x71' 'x121' 'x128' 'x132' 'x136' 'x140'
 'x143' 'x144' 'x147' 'x150' 'x160' 'x163' 'x164' 'x166' 'x192' 'x208'
 'x262' 'x272' 'x278']


In [26]:
print(scaled_df1.head())

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [10]:
print(X_selected.shape)

(181507, 25)


In [11]:
X_selected_df = pd.DataFrame(X_selected, columns=sfs.get_feature_names_out())

In [19]:
#keep only the selected features in the testone hot
test_onehot = scaled_df2[:, X_selected_df.columns.get_indexer(X_selected_df.columns)]
print(test_onehot.shape)

(77789, 25)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(scaled_df1, y, test_size = 0.3)

In [21]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
print("Training Set Info:")
print(X.info())

print("\nTest Set Info:")
print(test_onehot.info())


In [22]:
reg2 = LinearRegression().fit(X_selected, y)
print(reg2.score(X_selected, y))
print(reg2.coef_)
print(reg2.intercept_)
md_probs = reg2.predict(test_onehot)
#print("Mean squared error: %.2f" % mean_squared_error(test_onehot, md_probs))

0.6175966176602764
[[   51948.34519537    20555.50835646   690214.75138018  1413552.13165845
    154048.05221801   528410.3864093    436751.26382265   261673.58321836
    685524.55717093    91843.13571464   250471.20013476  1327398.68963339
   5085132.45619531   794670.60698325   871510.90735056   450779.2303655
    712278.30452995  3301107.2279228    165341.88380655   685867.78547406
    688912.86916235  1139660.40948749 -1619548.17690101 -1803980.03852777
  -1547832.2860476 ]]
[5831107.9284301]


In [23]:
additional_data = pd.read_csv("test.csv")

# Create a DataFrame with 'row ID' and predicted values
predicted_df = pd.DataFrame({
    'row ID': additional_data['row ID'],
    'price_doc': md_probs[:,0]  # Replace with the actual column name for your predictions
})

# Save the DataFrame to a new CSV file
predicted_df.to_csv('predictions11.csv', index=False)